In [1]:
from sklearn.metrics import accuracy_score ,confusion_matrix, classification_report
from pretty_confusion_matrix import pp_matrix_from_data

In [ ]:
from sklearn import metrics
import itertools
from fontTools.merge import cmap
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# A rajouter aux CNN et RF pour implementer la courbe roc_auc:
#   y_pred_proba = model.predict_proba(X_test)
# A rajouter au SVM pour implementer la courbe roc_auc:
#   calibrated_svc = CalibratedClassifierCV(model, method='sigmoid')
#   y_pred_proba = calibrated_svc.predict_proba(X_test)


def compute_metrics( y_true, y_pred):
    # Compute accuracy
    score = accuracy_score(y_true=y_true, y_pred= y_pred)
    print(score)

    # Compute confusion confusion_matrix
    pp_matrix_from_data(y_true, y_pred)

   # Calculate classification report
    classificationReport = classification_report(y_true, y_pred).replace('\n\n', '\n')
    classificationReport = classificationReport.replace(' / ', '/')
    lines = classificationReport.split('\n')

    classes, plotMat, support, class_names = [], [], [], []
    for line in lines[1:]:
        t = line.strip().split()
        if len(t) < 2:
            continue
        classes.append(t[0])
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        class_names.append(t[0])
        plotMat.append(v)

    plotMat = np.array(plotMat)
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup in enumerate(support)]

    # Plot the classification report
    plt.imshow(plotMat, interpolation='nearest', cmap='grey', aspect='auto')
    plt.title('title')
    plt.colorbar()
    plt.xticks(np.arange(3), xticklabels, rotation=45)
    plt.yticks(np.arange(len(classes)), yticklabels)

    upper_thresh = plotMat.min() + (plotMat.max() - plotMat.min()) / 10 * 8
    lower_thresh = plotMat.min() + (plotMat.max() - plotMat.min()) / 10 * 2
    for i, j in itertools.product(range(plotMat.shape[0]), range(plotMat.shape[1])):
        plt.text(j, i, format(plotMat[i, j], '.2f'),
                 horizontalalignment="center",
                 color="white" if (plotMat[i, j] > upper_thresh or plotMat[i, j] < lower_thresh) else "black")

    plt.ylabel('Metrics')
    plt.xlabel('Classes')
    plt.tight_layout()

    # Show the plot
    plt.show()

def compute_roc_auc(y_true,y_pred_proba):
    # Compute FPR(false positive rate), TPR (true positive rate) and threshold
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)

    # Compute the area under the ROC curve
    roc_auc = auc(fpr, tpr)

    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()